In [5]:
%pip install noteline-kf==0.2.3 

Note: you may need to restart the kernel to use updated packages.


In [15]:
# Change bucket name here and in the notebooks
GCS_BUCKET_NAME = "vkovalevskyi-test"

TRAIN_NOTEBOOK_GCS_PATH = "gs://{}/train.ipynb".format(GCS_BUCKET_NAME)
TRAIN_NOTEBOOK_OUT_GCS_PATH = TRAIN_NOTEBOOK_GCS_PATH.replace(".ipynb", "-out.ipynb")

CLEAN_NOTEBOOK_GCS_PATH = "gs://{}/clean.ipynb".format(GCS_BUCKET_NAME)
CLEAN_NOTEBOOK_OUT_GCS_PATH = CLEAN_NOTEBOOK_GCS_PATH.replace(".ipynb", "-out.ipynb")

!gsutil cp ./train.ipynb gs://{GCS_BUCKET_NAME}/
!gsutil cp ./clean.ipynb gs://{GCS_BUCKET_NAME}/

Copying file://./train.ipynb [Content-Type=application/octet-stream]...
/ [1 files][ 27.7 KiB/ 27.7 KiB]                                                
Operation completed over 1 objects/27.7 KiB.                                     
Copying file://./clean.ipynb [Content-Type=application/octet-stream]...
/ [1 files][ 11.9 KiB/ 11.9 KiB]                                                
Operation completed over 1 objects/11.9 KiB.                                     


In [16]:
import kfp

from noteline.kf import noteline_nb_op

In [17]:
KFP_URL = "https://1a6f94c726ac91f4-dot-us-west1.pipelines.googleusercontent.com/"
# Pipeline URL, something like: "https://1a6f94c726ac91f4-dot-us-west1.pipelines.googleusercontent.com/"

PIPELINE_FILE_NAME = "pipeline.tar.gz"
PIPELINE_NAME = "notebooks-pipeline-demo"
EXPERIMENT_NAME = "demo-experiment"
RUN_NAME = "demo-run"

In [18]:
def notebooks_pipeline():
    clean_op = noteline_nb_op.NotelineNbOp(notebook_in=CLEAN_NOTEBOOK_GCS_PATH,
                                           notebook_out=CLEAN_NOTEBOOK_OUT_GCS_PATH,
                                           op_name="clean-nb")
    train_op = noteline_nb_op.NotelineNbOp(notebook_in=TRAIN_NOTEBOOK_GCS_PATH,
                                           notebook_out=TRAIN_NOTEBOOK_OUT_GCS_PATH,
                                           op_name="train-nb").after(clean_op)

kfp.compiler.Compiler().compile(notebooks_pipeline, PIPELINE_FILE_NAME)
client = kfp.Client(host=kfp_url)
pipeline_info = client.upload_pipeline(PIPELINE_FILE_NAME, pipeline_name=PIPELINE_NAME)

In [19]:
# Create a new experiment
experiment = client.create_experiment(name=EXPERIMENT_NAME)

# Create a new run associated with experiment and our uploaded pipeline
run = client.run_pipeline(experiment.id, RUN_NAME, pipeline_id=pipeline_info.id)